In [40]:
import pandas as pd 
import pandasql as ps
import plotly.express as px
import plotly.graph_objects as go

from pandas import Series
from plotly.graph_objects import Figure

# Загрузка данных из источника

In [41]:
SRC_FOLDER = 'src'
FILE_NAME = 'data.tsv'

In [42]:
path_of_file = "\\".join([SRC_FOLDER, FILE_NAME])
df = pd.read_csv(path_of_file, sep='\t', header=None, names=["Query", "Timestamp", "Type"])
df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='s')
df

,Query,Timestamp,Type
0,малевич картины,2021-09-16 15:34:25,desktop
1,психология,2021-09-16 08:39:43,touch
2,с днём рождения лена,2021-09-16 05:52:43,touch
3,зверополис фильмы,2021-09-16 10:19:59,touch
4,алабай собака фото,2021-09-16 10:04:05,touch
...,...,...,...
1114360,бабочка,2021-09-12 19:40:53,touch
1114361,все республики россии,2021-09-12 16:08:54,touch
1114362,кухня в стиле лофт,2021-09-12 07:37:32,touch
1114363,топор,2021-09-12 12:22:27,desktop


# 1. Диапазоны дат

Укажите даты диапазона, предоставленного вам для анализа

In [43]:
sql_count_query_by_day = '''
select DATE(Timestamp) as query_date, COUNT(Timestamp) as cnt_query
from df
group By DATE(Timestamp)
'''

sql_count_by_day = ps.sqldf(sql_count_query_by_day, locals())

In [44]:
display(sql_count_by_day.head(10))
figure_query_by_day = px.line(sql_count_by_day, x="query_date", y="cnt_query")

figure_query_by_day.update_layout(
    xaxis_title="День",
    yaxis_title="Кол-во запросов",
    title="Кол-во запросов по дням"
)

figure_query_by_day

,query_date,cnt_query
0,2021-08-31,2773
1,2021-09-01,60957
2,2021-09-02,48999
3,2021-09-03,49734
4,2021-09-04,44458
5,2021-09-05,49300
6,2021-09-06,52675
7,2021-09-07,53394
8,2021-09-08,57200
9,2021-09-09,55181


In [45]:
sql_count_by_week = sql_count_by_day.reset_index()
sql_count_by_week["query_date"] = pd.to_datetime(sql_count_by_week["query_date"])
sql_count_by_week = sql_count_by_week.groupby(pd.Grouper(key="query_date", freq="7D", label='left')).sum().reset_index()
display(sql_count_by_week.head(10))
figure_grouped = px.bar(sql_count_by_week, x="query_date", y="cnt_query")
figure_grouped.update_layout(
    xaxis_title="Диапазоны дат",
    yaxis_title="Кол-во запросов",
    title="Распределение запросов по диапазонам"
)
figure_grouped

,query_date,index,cnt_query
0,2021-08-31,21,308896
1,2021-09-07,70,368940
2,2021-09-14,119,376176
3,2021-09-21,21,60353


# 2. Фильтрация

Рассчитайте кол-во запросов с текстом "ютуб"  в каждой платформе (desktop, touch)

In [46]:
sql_count_query_with_youtube = '''
    select * 
    from df
    where LOWER(query) like LOWER('%ютуб%') 
'''

df_count_query_with_youtube = ps.sqldf(sql_count_query_with_youtube, env=locals())
df_count_query_with_youtube.head(10)

,Query,Timestamp,Type
0,ютуб,2021-09-10 10:16:12.000000,desktop
1,ютуб youtube,2021-09-19 07:22:25.000000,touch
2,ютуб,2021-09-14 03:55:12.000000,touch
3,ютуб,2021-09-14 14:00:21.000000,desktop
4,ютуб,2021-09-13 21:49:36.000000,desktop
5,ютуб,2021-09-16 07:18:40.000000,desktop
6,ютуб,2021-09-05 10:04:17.000000,touch
7,ютуб,2021-09-06 15:15:10.000000,desktop
8,ютуб,2021-09-07 19:25:42.000000,desktop
9,ютуб,2021-09-07 17:28:45.000000,touch


In [47]:
df_count_query_with_youtube_group_by_type = df_count_query_with_youtube[["Query", "Type"]].groupby("Type").count().reset_index()
display(df_count_query_with_youtube_group_by_type.head(10))
figure_bar_group_by_type = px.bar(df_count_query_with_youtube_group_by_type, x="Type", y="Query")

figure_bar_group_by_type.update_layout(
    xaxis_title = "Платформа",
    yaxis_title = "Кол-во запросов", 
    title = "Сравнение кол-ва запросов по типу платформы",
)

figure_bar_group_by_type

,Type,Query
0,desktop,802
1,touch,597


# 3. Сортировка

Выведите топ-10 самых частотных запросов в каждой платформе (desktop, touch). Какие отличия вы видите?

In [48]:
sql_count_by_popular = ''' 
    select [Query], [Type], COUNT(*) as cnt_qeury_by_type
    from df
    group by [Query], [Type]
    order by cnt_qeury_by_type DESC
'''
df_count_by_popular = ps.sqldf(sql_count_by_popular, env=locals())

display(df_count_by_popular[df_count_by_popular["Type"] == "desktop"].head(10))
display(df_count_by_popular[df_count_by_popular["Type"] == "touch"].head(10))

,Query,Type,cnt_qeury_by_type
6,календарь 2021,desktop,2804
13,таблица менделеева,desktop,2631
30,картинки,desktop,1647
34,английский алфавит,desktop,1293
38,обои на рабочий стол,desktop,1143
41,Одноклассники (социальная сеть),desktop,1116
52,таблица квадратов,desktop,877
53,алфавит,desktop,874
54,таблица умножения,desktop,867
57,карта мира,desktop,795


,Query,Type,cnt_qeury_by_type
0,с днём рождения женщине,touch,4903
1,с днём рождения,touch,3967
2,с днём рождения мужчине,touch,3623
3,с днем рождения,touch,3005
4,погода,touch,2840
5,игры,touch,2833
7,фильмы,touch,2746
8,новости,touch,2708
9,музыка,touch,2696
10,мода,touch,2648


**Выводы:**
- Из результата самых популярных запросов видно, что в выборке преобладают запросы с мобильных устройств. Это может быть связано с тем, что идет подсчет уникальных значений. При поиске с телефона запрос более компактный и менее уникальный;
- В первую четверку входят запросы тематики поздравлений. Это может быть связано с тем, что поздравления передаются с помощью смс и мессенджеров;
- Большое количество запросов календаря с компьютера может быть связан с тем, что календарь часто используется на работе в офисе и для планирования отдыха дома;
- Мобильные запросы фокусируются на развлечениях, а компьютерные — на учебе и организации задач.

# 4. Анализ тренда

Посмотрите, чем отличается трафик запросов в течении дня. Как можно объяснить отличие?

In [49]:
sql_fraffic_per_day = ''' 
    select * 
    from df
    where DATE(Timestamp) = '2021-09-05'
'''
df_fraffic_per_day = ps.sqldf(sql_fraffic_per_day)
df_fraffic_per_day.head(10)

,Query,Timestamp,Type
0,малярия,2021-09-05 18:52:58.000000,touch
1,фильмы,2021-09-05 07:12:05.000000,touch
2,фильмы,2021-09-05 19:34:18.000000,touch
3,фильмы,2021-09-05 16:29:08.000000,touch
4,здоровье мультики,2021-09-05 06:22:04.000000,touch
5,новости наклонилась,2021-09-05 13:28:11.000000,touch
6,спорт,2021-09-05 07:22:39.000000,touch
7,форд мустанг,2021-09-05 12:34:39.000000,touch
8,доброе утро картинки,2021-09-05 05:27:08.000000,touch
9,производственный календарь 2022,2021-09-05 11:43:51.000000,desktop


In [50]:
def add_trace_to_fig(fig: Figure, x: Series, y: Series, title: str) -> Figure:
    fig.add_trace(go.Scatter(x=x, y=y, name=title))
    return fig

df_fraffic_per_day_hourly = df_fraffic_per_day.copy()
df_fraffic_per_day_hourly["Timestamp"] = pd.DatetimeIndex(df_fraffic_per_day_hourly["Timestamp"])
df_fraffic_per_day_hourly = df_fraffic_per_day_hourly.groupby(["Type", pd.Grouper(key="Timestamp", freq="h")]).count().reset_index()
display(df_fraffic_per_day_hourly.head(10))
figure_fraffic_per_day_hourly = go.Figure()

add_trace_to_fig(figure_fraffic_per_day_hourly, df_fraffic_per_day_hourly["Timestamp"], df_fraffic_per_day_hourly[df_fraffic_per_day_hourly["Type"] == "touch"]["Query"], "touch")
add_trace_to_fig(figure_fraffic_per_day_hourly, df_fraffic_per_day_hourly["Timestamp"], df_fraffic_per_day_hourly[df_fraffic_per_day_hourly["Type"] == "desktop"]["Query"], "desktop")

figure_fraffic_per_day_hourly

,Type,Timestamp,Query
0,desktop,2021-09-05 00:00:00,89
1,desktop,2021-09-05 01:00:00,89
2,desktop,2021-09-05 02:00:00,136
3,desktop,2021-09-05 03:00:00,186
4,desktop,2021-09-05 04:00:00,274
5,desktop,2021-09-05 05:00:00,412
6,desktop,2021-09-05 06:00:00,543
7,desktop,2021-09-05 07:00:00,776
8,desktop,2021-09-05 08:00:00,939
9,desktop,2021-09-05 09:00:00,1028


**Выводы:**
- Кол-во запросов с мобильных устройств преобладает на момент 05-09-2021. Это может быть связано с тем, что телефон очень компактен и удобен в использовании для поиска информации и развлечений;
- Значительный рост запросов начинается с 04:00 и заканчивается в 20:00;
- Наблюдается равномерное кол-во запросов с 06:00 до 18:00, что может быть связано с пробуждением и дневной деятельностью людей;
- Пики в 07:00 и 17:00 могут быть связаны с тем, что в эти промежутки люди наиболее свободны от работы и домашних дел.

# 5. Темы запросов на разных платформах

Выделите тематики запросов, контрастные для мобильных и компьютеров - темы, доля которых отличаются на разных платформах.

Выделены следующие тема для рассмотрения: 18+, игры, образование, планирование, погода, поздравления. Каждой теме определены ключевые слова для поиска релевантных запросов.

In [51]:
df_topics = df.copy()
df_topics = df_topics[["Query", "Type"]]

# topic, keywords
dict_topics_keywords = {
    "Игры": ["игр", "battle royale", "minecraft"],
    "Планирование": ["план", "отпуск", "календар"],
    "Образование": ["образование", "курс", "сертификат", "урок", "алфавит", "наука", "таблица", "алфавит"],
    "Поздравления": ["поздравл", "с днем р", "открытка", "с добрым"],
    "Погода": ["прогноз п", "погода", "дождь", "температура", "осадки"],
    "18+": ["18+", "порн", "xxx", "яой", "boobs"],
}

figure_bar_topics = go.Figure()
df_topics["Topic"] = "Другое"

for topic, keywords in dict_topics_keywords.items():
    filer_rulle = "|".join(keywords)
    df_topics.loc[df_topics["Query"].str.contains(filer_rulle, case=False), "Topic"] = topic

df_topics_agg = df_topics[df_topics["Topic"] != "Другое"].groupby(["Type", "Topic"]).count().reset_index()
display(df_topics_agg)
figure_bar_topics = px.bar(df_topics_agg, x="Topic", y="Query", color="Type", barmode="group")

figure_bar_topics.update_layout(
    xaxis_title="Темы",
    yaxis_title="Кол-во запросов",
    title="Темы в рамках платформ"
)

figure_bar_topics

,Type,Topic,Query
0,desktop,18+,640
1,desktop,Игры,2794
2,desktop,Образование,17476
3,desktop,Планирование,11142
4,desktop,Погода,1130
5,desktop,Поздравления,4125
6,touch,18+,2984
7,touch,Игры,6605
8,touch,Образование,23516
9,touch,Планирование,7829


**Выводы:**
- В выборке по темам преобладают запросы с мобильных устройств, что связано с их растущей популярностью, компактностью и удобством в использовании;
- Большое количество запросов приходится на тематику поздравлений т.к. люди в большей степени контактируют через смс и мессенджеры;
- Единственная рассмотренная тема, где преобладают запросы с компьютера - Планирование. Это может быть связано с тем, что люди стараются тщательно планировать свою деятельность (работу, отдых);
- Результаты анализа сильно зависят от подобранных ключевых слов тематики. Для большей прозрачности, необходимо расширять перечень ключевых слов по темам.